In [ ]:
nodes_artists  
nodes_tags   
rel_artist_has_album  
rel_performs         
rel_similar_songs  
rel_song_in_album
nodes_albums   
nodes_songs    
nodes_years  
rel_artist_has_tag   
rel_similar_artists  
rel_song_has_tag   
rel_song_year


In [1]:
%%writefile scripts/rename-and-move.py
#!/usr/bin/env bash

folders=(nodes_artists nodes_songs nodes_albums nodes_years nodes_tags rel_similar_artists rel_performs rel_artist_has_album rel_artist_has_tag rel_song_in_album rel_similar_songs rel_song_has_tag rel_song_year)
newName=(artist song album year tag sim-a perf has-album a-has-tag in-album sim-s s-has-tag release)

# Rename files
for i in {0..12}
do 
rename part-00 ${newName[$i]}- /graph/import/tmp/${folders[$i]}/part*
done

for i in {0..12}
do 
cp /graph/import/tmp/${folders[$i]}/${newName[$i]}* /graph/neo4j/data/import/
done

# Create header CSV files
echo "id:ID(artist),idmb,id7d,name" > /graph/neo4j/data/import/hdr-artist.csv
echo "songid,id:ID(song),title,danceability:FLOAT,duration:FLOAT,energy:FLOAT,loudness:FLOAT" > /graph/neo4j/data/import/hdr-song.csv
echo "name:ID(album)" > /graph/neo4j/data/import/hdr-album.csv
echo "year:ID(year)" > /graph/neo4j/data/import/hdr-year.csv
echo "tag:ID(tag)" > /graph/neo4j/data/import/hdr-tag.csv
 
echo ":START_ID(artist),:END_ID(artist)" > /graph/neo4j/data/import/hdr-sim-a.csv
echo ":START_ID(artist),:END_ID(song)" > /graph/neo4j/data/import/hdr-perf.csv
echo ":START_ID(artist),:END_ID(album)" > /graph/neo4j/data/import/hdr-has-album.csv
echo ":START_ID(artist),:END_ID(tag),frq:FLOAT,weight:FLOAT" > /graph/neo4j/data/import/hdr-a-has-tag.csv
echo ":START_ID(song),:END_ID(album)" > /graph/neo4j/data/import/hdr-in-album.csv
echo ":START_ID(song),:END_ID(song),weight:FLOAT" > /graph/neo4j/data/import/hdr-sim-s.csv
echo ":START_ID(song),:END_ID(tag),weight:FLOAT" > /graph/neo4j/data/import/hdr-s-has-tag.csv
echo ":START_ID(song),:END_ID(year)" > /graph/neo4j/data/import/hdr-release.csv

Writing scripts/rename-and-move.py


In [2]:
!chmod a+x scripts/rename-and-move.py

In [3]:
!echo $NEO4J_HOME

/graph/neo4j/bin


In [25]:
%%writefile scripts/importToNeo.py
#!/usr/bin/env bash

cd /graph/neo4j/data/import

$NEO4J_HOME/neo4j-import --into /graph/neo4j/data/graph.db \
--skip-duplicate-nodes true --bad-tolerance 9000000 \
--id-type string \
--nodes:ARTIST hdr-artist.csv,$(echo $(ls art*) | tr ' ' ,)  \
--nodes:SONG   hdr-song.csv,$(echo $(ls song*) | tr ' ' ,)  \
--nodes:ALBUM  hdr-album.csv,$(echo $(ls album*) | tr ' ' ,)  \
--nodes:YEAR   hdr-year.csv,$(echo $(ls year*) | tr ' ' ,)  \
--nodes:TAG    hdr-tag.csv,$(echo $(ls tag*) | tr ' ' ,)  \
--relationships:SIMILAR_TO  hdr-sim-a.csv,$(echo $(ls sim-a*) | tr ' ' ,)  \
--relationships:PERFORMS    hdr-perf.csv,$(echo $(ls perf*) | tr ' ' ,)  \
#--relationships:HAS_ALBUM   hdr-has-album.csv,$(echo $(ls has-album*) | tr ' ' ,)  \
#--relationships:HAS_TAG     hdr-a-has-tag.csv,$(echo $(ls a-has-tag*) | tr ' ' ,)  \
#--relationships:IN_ALBUM    hdr-in-album.csv,$(echo $(ls in-album*) | tr ' ' ,)  \
#--relationships:SIMILAR_TO  hdr-sim-s.csv,$(echo $(ls sim-s*) | tr ' ' ,)  \
#--relationships:HAS_TAG     hdr-s-has-tag.csv,$(echo $(ls s-has-tag*) | tr ' ' ,)  \
#--relationships:RELEASED_ON hdr-release.csv,$(echo $(ls release*) | tr ' ' ,) 

Overwriting scripts/importToNeo.py


In [11]:
!chmod a+x scripts/importToNeo.py

In [26]:
!time scripts/importToNeo.py

Importing the contents of these files into /graph/neo4j/data/graph.db:
Nodes:
  :ARTIST
  /graph/neo4j/data/import/hdr-artist.csv
  /graph/neo4j/data/import/artist-000
  /graph/neo4j/data/import/artist-001
  /graph/neo4j/data/import/artist-002
  /graph/neo4j/data/import/artist-003
  /graph/neo4j/data/import/artist-004
  /graph/neo4j/data/import/artist-005
  /graph/neo4j/data/import/artist-006
  /graph/neo4j/data/import/artist-007
  /graph/neo4j/data/import/artist-008
  /graph/neo4j/data/import/artist-009
  /graph/neo4j/data/import/artist-010
  /graph/neo4j/data/import/artist-011
  /graph/neo4j/data/import/artist-012
  /graph/neo4j/data/import/artist-013
  /graph/neo4j/data/import/artist-014
  /graph/neo4j/data/import/artist-015
  /graph/neo4j/data/import/artist-016
  /graph/neo4j/data/import/artist-017
  /graph/neo4j/data/import/artist-018
  /graph/neo4j/data/import/artist-019
  /graph/neo4j/data/import/artist-020
  /graph/neo4j/data/import/artist-021
  /graph/neo4j/data/import/artist-

In [29]:
!tail /graph/neo4j/data/graph.db/bad.log

   startNode: AR6OEO01187B99A5E8
   endNode: ARPDZCX11EBCD780E7
   type: SIMILAR_TO
 refering to missing node ARPDZCX11EBCD780E7
InputRelationship:
   source: /graph/neo4j/data/import/sim-a-031:123758
   startNode: AR7Y8171187B989544
   endNode: ARX60GE11C8A42B07F
   type: SIMILAR_TO
 refering to missing node ARX60GE11C8A42B07F


In [ ]:
$NEO4J_HOME/bin/neo4j-import --into /graph/neo4j/data/testyear.db \
--skip-duplicate-nodes true --bad-tolerance 90000 \
--id-type string \
--nodes:Artist hdr-artist.csv,$(echo $(ls art*) | tr ' ' ,)